Alpha Vantage API KEY: R9072PNN6E5D0TZC

In [45]:
import requests
import pandas as pd
import yfinance as yf
import numpy as np
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [16]:
symbol = 'AAPL'

In [25]:
data = yf.download(symbol, start="1980-12-12", end="2023-10-26", interval='1d')

[*********************100%***********************]  1 of 1 completed


In [31]:
data.reset_index(inplace=True)

In [35]:
data

Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
0,1980-12-12 00:00:00+00:00,0.098943,0.128348,0.128906,0.128348,0.128348,469033600
1,1980-12-15 00:00:00+00:00,0.093781,0.121652,0.122210,0.121652,0.122210,175884800
2,1980-12-16 00:00:00+00:00,0.086898,0.112723,0.113281,0.112723,0.113281,105728000
3,1980-12-17 00:00:00+00:00,0.089049,0.115513,0.116071,0.115513,0.115513,86441600
4,1980-12-18 00:00:00+00:00,0.091630,0.118862,0.119420,0.118862,0.118862,73449600
...,...,...,...,...,...,...,...
10803,2023-10-19 00:00:00+00:00,174.566727,175.460007,177.839996,175.190002,176.039993,59302900
10804,2023-10-20 00:00:00+00:00,171.999863,172.880005,175.419998,172.639999,175.309998,64189300
10805,2023-10-23 00:00:00+00:00,172.119247,173.000000,174.009995,169.929993,170.910004,55980100


In [36]:
model_data = data[['Date', 'Close']]

In [37]:
model_data['Close_next'] = data['Close'].shift(-1)
model_data['Close_1d_ago'] = data['Close'].shift(1)
model_data['Close_2d_ago'] = data['Close'].shift(2)
model_data['Close_3d_ago'] = data['Close'].shift(3)

C:\Users\Vashist\AppData\Local\Temp\ipykernel_13540\3027388394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['Close_next'] = data['Close'].shift(-1)
C:\Users\Vashist\AppData\Local\Temp\ipykernel_13540\3027388394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['Close_1d_ago'] = data['Close'].shift(1)
C:\Users\Vashist\AppData\Local\Temp\ipykernel_13540\3027388394.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [39]:
model_data.dropna(inplace=True)

C:\Users\Vashist\AppData\Local\Temp\ipykernel_13540\3310783176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.dropna(inplace=True)


In [40]:
model_data

Price,Date,Close,Close_next,Close_1d_ago,Close_2d_ago,Close_3d_ago
Ticker,,AAPL,,,,
3,1980-12-17 00:00:00+00:00,0.115513,0.118862,0.112723,0.121652,0.128348
4,1980-12-18 00:00:00+00:00,0.118862,0.126116,0.115513,0.112723,0.121652
5,1980-12-19 00:00:00+00:00,0.126116,0.132254,0.118862,0.115513,0.112723
6,1980-12-22 00:00:00+00:00,0.132254,0.137835,0.126116,0.118862,0.115513
7,1980-12-23 00:00:00+00:00,0.137835,0.145089,0.132254,0.126116,0.118862
...,...,...,...,...,...,...
10802,2023-10-18 00:00:00+00:00,175.839996,175.460007,177.149994,178.720001,178.850006
10803,2023-10-19 00:00:00+00:00,175.460007,172.880005,175.839996,177.149994,178.720001
10804,2023-10-20 00:00:00+00:00,172.880005,173.000000,175.460007,175.839996,177.149994


In [41]:
X = model_data[['Close', 'Close_1d_ago', 'Close_2d_ago', 'Close_3d_ago']]
y = model_data['Close_next']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [43]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 7, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4]
}

In [46]:
xgb_model = XGBRegressor(objective='reg:squarederror')
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid,
    n_iter=50, scoring='neg_mean_absolute_error', cv=3, verbose=1, random_state=42, n_jobs=-1)

In [47]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6, 7, 8, 10],
                                        'n_estimators': [50, 100, 200, 300],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=1)

In [49]:
best_params = random_search.best_params_

In [50]:
best_score = -random_search.best_score_

In [52]:
print(f"Best Parameters: {best_params}")

Best Parameters: {'subsample': 0.7, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.2, 'gamma': 0, 'colsample_bytree': 0.6}


In [53]:
print(f"Best Score: {best_score}")

Best Score: 0.658235313863774


In [54]:
best_model = XGBRegressor(**best_params, objective='reg:squarederror')

In [55]:
best_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [56]:
y_pred = best_model.predict(X_test)

In [61]:
mae = mean_absolute_error(y_test, y_pred)

In [62]:
mae

50.31513335747457